In [9]:
import aoc
import re

In [6]:
def pairs_in_template(polymer_template):
    for idx in range(len(polymer_template) - 1):
        print(polymer_template[idx:idx + 2])

In [7]:
pairs_in_template('NNCB')

NN
NC
CB


In [8]:
example_insertion_rules = r'''CH -> B
HH -> N
CB -> H
NH -> C
HB -> C
HC -> B
HN -> C
NN -> C
BH -> H
NC -> B
NB -> B
BN -> B
BB -> N
BC -> B
CC -> N
CN -> C'''

In [12]:
def insertion_rules_dict(insertion_rules_string):
    insertion_rules = re.findall(r'(\w\w) -> (\w)', insertion_rules_string)
    return {r[0]: r[1] for r in insertion_rules}

In [14]:
example_rules = insertion_rules_dict(example_insertion_rules)

{'CH': 'B',
 'HH': 'N',
 'CB': 'H',
 'NH': 'C',
 'HB': 'C',
 'HC': 'B',
 'HN': 'C',
 'NN': 'C',
 'BH': 'H',
 'NC': 'B',
 'NB': 'B',
 'BN': 'B',
 'BB': 'N',
 'BC': 'B',
 'CC': 'N',
 'CN': 'C'}

In [23]:
def apply_rules_to_template(polymer_template, insertion_rules):
    polymer_entries = []
    for idx in range(len(polymer_template) - 1):
        polymer_entries.append((idx, polymer_template[idx]))
        pair = polymer_template[idx:idx + 2]
        if pair in insertion_rules:
            polymer_entries.append((idx + 0.5, insertion_rules[pair]))
    polymer_entries.append((idx + 1, polymer_template[-1]))
    polymer_entries.sort()
    return ''.join([p[1] for p in polymer_entries])

In [24]:
apply_rules_to_template('NNCB', example_rules)

'NCNBCHB'

In [25]:
apply_rules_to_template('NCNBCHB', example_rules)

'NBCCNBBBCBHCB'

In [36]:
def polymerise(polymer_template, insertion_rules, num_steps):
    for k in range(num_steps):
        polymer_template = apply_rules_to_template(polymer_template, insertion_rules)
    return polymer_template

In [37]:
polymerise('NNCB', example_rules, 2)

'NBCCNBBBCBHCB'

In [38]:
polymer10 = polymerise('NNCB', example_rules, 10)

In [40]:
assert len(polymer10) == 3073

In [44]:
element_counts = aoc.distinct_counts(polymer10)
most_common_element = aoc.most_common_entry(polymer10)
least_common_element = aoc.least_common_entry(polymer10)

In [46]:
assert element_counts[most_common_element] == 1749
assert element_counts[least_common_element] == 161

In [47]:
day14_string = aoc.read_file_as_string('inputs/day14.txt')

In [48]:
day14_rules = insertion_rules_dict(day14_string)

In [50]:
len(day14_rules)

100

In [52]:
day14_polymer_template = aoc.read_file_as_list('inputs/day14.txt')[0]

In [53]:
day14_polymer_template

'FSKBVOSKPCPPHVOPVFPC'

In [54]:
polymer10 = polymerise(day14_polymer_template, day14_rules, 10)

In [55]:
element_counts = aoc.distinct_counts(polymer10)
most_common_element = aoc.most_common_entry(polymer10)
least_common_element = aoc.least_common_entry(polymer10)

In [56]:
element_counts[most_common_element] - element_counts[least_common_element]

2360

In [58]:
# star 2

In [59]:
polymer10 = polymerise('NNCB', example_rules, 10)

In [61]:
polymer10a = polymerise('NN', example_rules, 10)
polymer10b = polymerise('NC', example_rules, 10)
polymer10c = polymerise('CB', example_rules, 10)

In [71]:
alt_route = polymer10a + polymer10b[1:-1] + polymer10c

In [66]:
len(polymer10)

3073

In [72]:
len(alt_route)

3073

In [73]:
alt_route == polymer10

True

In [80]:
%time x=polymerise('FS', day14_rules, 24)

Wall time: 12.2 s


In [81]:
2**16

65536

In [89]:
def direct_count_for_pair(pair, insertion_rules, num_steps):
    polymer = polymerise(pair, insertion_rules, num_steps)
    return aoc.distinct_counts(polymer)

In [90]:
direct_count_for_pair('FS', day14_rules, 1)

{'F': 1, 'C': 1, 'S': 1}

In [84]:
def add_dicts(x, y):
    return {k: x.get(k, 0) + y.get(k, 0) for k in set(x) | set(y)}

In [88]:
counts1 = aoc.distinct_counts(polymerise('FSK', day14_rules, 1))
counts1

{'F': 1, 'C': 1, 'S': 1, 'O': 1, 'K': 1}

In [91]:
direct_count_for_pair('FS', day14_rules, 1)

{'F': 1, 'C': 1, 'S': 1}

In [92]:
direct_count_for_pair('SK', day14_rules, 1)

{'S': 1, 'O': 1, 'K': 1}

In [93]:
add_dicts(direct_count_for_pair('FS', day14_rules, 1), direct_count_for_pair('SK', day14_rules, 1))

{'K': 1, 'S': 2, 'F': 1, 'O': 1, 'C': 1}

In [95]:
def recurse_counts_for_pair(pair, insertion_rules, num_steps):
    if num_steps <= 10:
        polymer = polymerise(pair, insertion_rules, num_steps)
        return aoc.distinct_counts(polymer)
    elif pair not in insertion_rules:
        return {pair[0]: 1, pair[1]:1}
    else:
        insert = insertion_rules[pair]
        counts_left = recurse_counts_for_pair(pair[0]+insert, insertion_rules, num_steps - 1)
        counts_right = recurse_counts_for_pair(insert+pair[1], insertion_rules, num_steps - 1)
        counts = add_dicts(counts_left, counts_right)
        counts[insert] -= 1
        return counts

In [96]:
%time recurse_counts_for_pair('FS', day14_rules, 10)

Wall time: 999 µs


{'F': 83,
 'P': 203,
 'H': 105,
 'V': 132,
 'B': 116,
 'S': 59,
 'O': 113,
 'N': 120,
 'K': 38,
 'C': 56}

In [97]:
%time recurse_counts_for_pair('FS', day14_rules, 11)

Wall time: 2 ms


{'B': 234,
 'N': 231,
 'V': 281,
 'O': 257,
 'F': 161,
 'P': 385,
 'S': 120,
 'C': 99,
 'H': 197,
 'K': 84}

In [98]:
%time recurse_counts_for_pair('FS', day14_rules, 12)

Wall time: 4 ms


{'B': 475,
 'N': 465,
 'V': 584,
 'F': 317,
 'O': 542,
 'P': 737,
 'S': 260,
 'C': 189,
 'H': 370,
 'K': 158}

In [114]:
%time recurse_counts_for_pair('FS', day14_rules, 25)

Wall time: 25.2 s


{'B': 3560991,
 'N': 3764029,
 'V': 5909474,
 'O': 5729863,
 'F': 2605860,
 'P': 4843284,
 'S': 2082078,
 'C': 1360451,
 'H': 2348525,
 'K': 1349878}

In [115]:
%time aoc.distinct_counts(polymerise('FS', day14_rules, 25))

Wall time: 34 s


{'F': 2605860,
 'P': 4843284,
 'H': 2348525,
 'V': 5909474,
 'B': 3560991,
 'S': 2082078,
 'O': 5729863,
 'N': 3764029,
 'K': 1349878,
 'C': 1360451}

In [122]:
class Polymerizer():
    def __init__(self, insertion_rules_string):
        insertion_rules = re.findall(r'(\w\w) -> (\w)', insertion_rules_string)
        self.insertion_rules = {r[0]: r[1] for r in insertion_rules}
    
    def apply_rules_to_template(self, polymer_template):
        polymer_entries = []
        for idx in range(len(polymer_template) - 1):
            polymer_entries.append((idx, polymer_template[idx]))
            pair = polymer_template[idx:idx + 2]
            if pair in self.insertion_rules:
                polymer_entries.append((idx + 0.5, self.insertion_rules[pair]))
        polymer_entries.append((idx + 1, polymer_template[-1]))
        polymer_entries.sort()
        return ''.join([p[1] for p in polymer_entries])
    
    def polymerize(self, polymer_template, num_steps):
        for k in range(num_steps):
            polymer_template = self.apply_rules_to_template(polymer_template)
        return polymer_template
    
    def counts_after_polymerization(self, polymer_template, num_steps):
        return aoc.distinct_counts(self.polymerize(polymer_template, num_steps))
    
    def score_after_polymerization(self, polymer_template, num_steps):
        element_counts = self.counts_after_polymerization(polymer_template, num_steps)
        ordered_counts = sorted([(v, k) for k, v in element_counts.items()])
        return ordered_counts[-1][0] - ordered_counts[0][0]
    

In [123]:
example_polymerizer = Polymerizer(example_insertion_rules)

In [124]:
example_polymerizer.score_after_polymerization('NNCB', 10)

1588

In [126]:
full_polymerizer = Polymerizer(day14_string)

In [128]:
full_polymerizer.score_after_polymerization(day14_polymer_template, 10)

2360

In [144]:
class RecursivePolymerizer():
    def __init__(self, insertion_rules_string):
        insertion_rules = re.findall(r'(\w\w) -> (\w)', insertion_rules_string)
        self.insertion_rules = {r[0]: r[1] for r in insertion_rules}
    
    def apply_rules_to_template(self, polymer_template):
        polymer_entries = []
        for idx in range(len(polymer_template) - 1):
            polymer_entries.append((idx, polymer_template[idx]))
            pair = polymer_template[idx:idx + 2]
            if pair in self.insertion_rules:
                polymer_entries.append((idx + 0.5, self.insertion_rules[pair]))
        polymer_entries.append((idx + 1, polymer_template[-1]))
        polymer_entries.sort()
        return ''.join([p[1] for p in polymer_entries])
    
    def polymerize(self, polymer_template, num_steps):
        for k in range(num_steps):
            polymer_template = self.apply_rules_to_template(polymer_template)
        return polymer_template
    
    def counts_after_polymerization_of_pair(self, pair, num_steps):
        return aoc.distinct_counts(self.polymerize(pair, num_steps))  # TODO: something smarter here
        
    def counts_after_polymerization(self, polymer_template, num_steps):
        scores = self.counts_after_polymerization_of_pair(polymer_template[:2], num_steps)
        scores[polymer_template[1]] -= 1
        for idx in range(1, len(polymer_template) - 1):
            scores_idx = self.counts_after_polymerization_of_pair(polymer_template[idx: idx + 2], num_steps)
            scores_idx[polymer_template[idx + 1]] -= 1
            scores = add_dicts(scores, scores_idx)
        scores[polymer_template[-1]] += 1
        return scores
        
    

In [149]:
example_polymerizer.counts_after_polymerization('NNCB', 10)

{'N': 865, 'B': 1749, 'C': 298, 'H': 161}

In [145]:
example_polymerizer2 = RecursivePolymerizer(example_insertion_rules)

In [150]:
example_polymerizer2.counts_after_polymerization('NNCB', 10)

{'H': 161, 'B': 1749, 'C': 298, 'N': 865}